dataset balance check

In [1]:
import pandas as pd

data = pd.read_excel("student_embeddings_output.xlsx")
low = data[data["Numerical_Label"] == 0]
med = data[data["Numerical_Label"] == 1]
high = data[data["Numerical_Label"] == 2]

print(len(low))
print(len(med))
print(len(high))

463
1290
1089


balancing dataset


In [ ]:
import pandas as pd
from imblearn.over_sampling import SMOTE


# Drop non-numeric columns (like 'Student' and 'Teacher')
X = data.drop(columns=['Student_Text', 'Teacher_Text', 'Fluency_Label'])

# Convert column names to strings
X.columns = X.columns.astype(str)

# Target column
y = data['Numerical_Label']

# Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Create DataFrame from resampled data
X_resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
y_resampled_df = pd.DataFrame(y_resampled, columns=['Label'])

# Combine into one DataFrame
balanced_df = pd.concat([X_resampled_df, y_resampled_df], axis=1)

# Save to Excel
balanced_df.to_excel("final_student_embeddings_upsampled.xlsx", index=False)
print("Upsampled dataset saved successfully.")

checking balance


In [ ]:
import pandas as pd

data = pd.read_excel("final_student_embeddings_upsampled.xlsx")
low = data[data["Numerical_Label"] == 0]
med = data[data["Numerical_Label"] == 1]
high = data[data["Numerical_Label"] == 2]

print(len(low))
print(len(med))
print(len(high))

1290
1290
1290


NCA and Lime

In [3]:
pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 24.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=f62693cc6d227d99968728792c656e08ab5757a81bc5505029daaa5c5198aa2a
  Stored in directory: /root/.cache/pip/wheels/85/fa/a3/9c2d44c9f3cd77cf4e533b58900b2bf4487f2a17e8ec212a3d
Successfully built lime


In [4]:
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import pandas as pd
import numpy as np

# Optional: XGBoost
try:
    from xgboost import XGBClassifier
    xgb_available = True
except ImportError:
    xgb_available = False

# Load updated dataset
df = pd.read_excel("final_student_embeddings_upsampled.xlsx")

# Update this column name if needed
target_column = "Numerical_Label"

# Features and Labels
X = df.iloc[:, :768].values
y = df[target_column]
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# NCA
nca = NeighborhoodComponentsAnalysis(n_components=2, random_state=42)
X_nca = nca.fit_transform(X, y_encoded)

# Analyze top contributing original features to NCA components
nca_components = nca.components_  # Shape: (2, 768)
top_k = 10  # Top N contributing features to show

for i in range(2):
    component = nca_components[i]
    top_indices = np.argsort(np.abs(component))[::-1][:top_k]
    print(f"\n🔍 Top {top_k} original features contributing to NCA Component {i + 1}:")
    for idx in top_indices:
        print(f"Feature {idx}: Weight = {component[idx]:.4f}")

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_nca)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# Define parameter grids for hyperparameter tuning
param_grids = {
    "Logistic Regression": {
        'C': [0.01, 0.1, 1, 10, 100],
        'solver': ['lbfgs', 'liblinear']
    },
    "SVM": {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf']
    },
    "Random Forest": {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 5, 10],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 5]
    },
    "Naive Bayes": {},
    "Decision Tree": {
        'max_depth': [None, 5, 10],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 5]
    },
    "AdaBoost": {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 1.0]
    },
    "MLP": {
        'hidden_layer_sizes': [(50,), (100,), (100, 50)],
        'activation': ['relu', 'tanh'],
        'learning_rate': ['constant', 'adaptive']
    },
    "KNN": {
        'n_neighbors': list(range(1, 21)),
        'weights': ['uniform', 'distance']
    }
}

if xgb_available:
    param_grids["XGBoost"] = {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.3]
    }

# Base models
base_models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(probability=True, random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Naive Bayes": GaussianNB(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "MLP": MLPClassifier(max_iter=500, random_state=42),
    "KNN": KNeighborsClassifier()
}

if xgb_available:
    base_models["XGBoost"] = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Hyperparameter tuning
best_models = {}
for name, model in base_models.items():
    if param_grids[name]:
        print(f"Tuning {name}...")
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grids[name],
            cv=5,
            scoring='accuracy',
            n_jobs=-1
        )
        grid_search.fit(X_train, y_train)
        best_models[name] = grid_search.best_estimator_
        print(f"Best parameters for {name}: {grid_search.best_params_}")
    else:
        model.fit(X_train, y_train)
        best_models[name] = model
        print(f"No tuning required for {name}")

# Stacking classifier
stacking_estimators = [(name.lower().replace(" ", "_"), model) for name, model in best_models.items()]
stacking_clf = StackingClassifier(
    estimators=stacking_estimators,
    final_estimator=LogisticRegression(max_iter=1000, random_state=42),
    cv=5
)

# Add stacking
models = best_models.copy()
models["Stacking (All Models)"] = stacking_clf

# Evaluate
for name, model in models.items():
    print(f"📌 {name}")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    train_acc = accuracy_score(y_train, model.predict(X_train))
    test_acc = accuracy_score(y_test, y_pred)

    print(f"Training Accuracy: {train_acc:.4f}")
    print(f"Testing Accuracy:  {test_acc:.4f}")
    print("Classification Report:")
    target_names = [str(label) for label in label_encoder.classes_]
    print(classification_report(y_test, y_pred, target_names=target_names))
    print("-" * 60)


from lime.lime_tabular import LimeTabularExplainer
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt  # <-- Add this import

# Prepare the test set in original feature space
_, X_test_original, _, y_test_original = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)
instance_original = X_test_original[0]

# Set up LIME explainer for original features
explainer = LimeTabularExplainer(
    training_data=X,
    feature_names=[f"emb_{i}" for i in range(X.shape[1])],
    class_names=[str(label) for label in label_encoder.classes_],
    mode='classification',
    random_state=42,
    discretize_continuous=False
)

print("\nLIME Explanations for Test Instance (First Sample, original features):")
print(f"Instance values: {instance_original}")
print("Predicted class for each model:")

for name, model in models.items():
    # Skip stacking for LIME unless you want advanced handling
    if name == "Stacking (All Models)":
        print(f"{name}: Skipped LIME explanation (complex pipeline)")
        continue

    # Build pipeline: StandardScaler -> NCA -> Model (for LIME only)
    lime_pipeline = make_pipeline(
        StandardScaler(),
        NeighborhoodComponentsAnalysis(n_components=2, random_state=42),
        model
    )
    # Fit pipeline on ALL original data (NCA is supervised)
    lime_pipeline.fit(X, y_encoded)

    pred = lime_pipeline.predict([instance_original])[0]
    print(f"{name}: {label_encoder.inverse_transform([pred])[0]}")

    print(f"\nExplaining predictions for {name}...")
    try:
        explanation = explainer.explain_instance(
            data_row=instance_original,
            predict_fn=lime_pipeline.predict_proba,
            num_features=10,  # Show top 10 original features
            num_samples=5000
        )
        # Save as HTML
        explanation.save_to_file(f"lime_explanation_{name.replace(' ', '_')}_origfeatures.html")
        # Save as image
        fig = explanation.as_pyplot_figure()
        plt.tight_layout()
        fig.savefig(f"lime_explanation_{name.replace(' ', '_')}_origfeatures.png")
        plt.close(fig)
    except Exception as e:
        print(f"Failed to generate explanation for {name}: {str(e)}")
    print("-" * 60)





🔍 Top 10 original features contributing to NCA Component 1:
Feature 21: Weight = 6.5062
Feature 614: Weight = 6.4133
Feature 622: Weight = 5.7168
Feature 247: Weight = 5.4134
Feature 241: Weight = 5.3150
Feature 296: Weight = 5.1558
Feature 747: Weight = -5.1285
Feature 499: Weight = 5.1238
Feature 9: Weight = 5.0146
Feature 258: Weight = 4.9513

🔍 Top 10 original features contributing to NCA Component 2:
Feature 441: Weight = 6.8877
Feature 236: Weight = 6.4547
Feature 96: Weight = -5.8187
Feature 549: Weight = 5.7652
Feature 416: Weight = 5.7212
Feature 14: Weight = -5.6918
Feature 421: Weight = -5.6676
Feature 261: Weight = 5.4887
Feature 45: Weight = 5.4108
Feature 340: Weight = -5.3644
Tuning Logistic Regression...
Best parameters for Logistic Regression: {'C': 0.01, 'solver': 'lbfgs'}
Tuning SVM...
Best parameters for SVM: {'C': 10, 'kernel': 'rbf'}
Tuning Random Forest...
Best parameters for Random Forest: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_e

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Best parameters for MLP: {'activation': 'tanh', 'hidden_layer_sizes': (100, 50), 'learning_rate': 'constant'}
Tuning KNN...
Best parameters for KNN: {'n_neighbors': 9, 'weights': 'uniform'}
Tuning XGBoost...


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:09:18] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best parameters for XGBoost: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
📌 Logistic Regression
Training Accuracy: 0.7930
Testing Accuracy:  0.8049
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.96      0.89       265
           1       0.80      0.68      0.73       260
           2       0.79      0.78      0.78       249

    accuracy                           0.80       774
   macro avg       0.80      0.80      0.80       774
weighted avg       0.80      0.80      0.80       774

------------------------------------------------------------
📌 SVM
Training Accuracy: 0.8317
Testing Accuracy:  0.8295
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.94      0.93       265
           1       0.74      0.80      0.77       260
           2       0.82      0.75      0.79       249

    accuracy                           0.83       774
   macro avg       

...

Training Accuracy: 0.8547
Testing Accuracy:  0.8501
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94       265
           1       0.78      0.82      0.80       260
           2       0.84      0.80      0.82       249

    accuracy                           0.85       774
   macro avg       0.85      0.85      0.85       774
weighted avg       0.85      0.85      0.85       774

------------------------------------------------------------
📌 KNN
Training Accuracy: 0.8534
Testing Accuracy:  0.8540
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.94      0.93       265
           1       0.78      0.83      0.81       260
           2       0.85      0.79      0.82       249

    accuracy                           0.85       774
   macro avg       0.85      0.85      0.85       774
weighted avg       0.85      0.85      0.85       774

---------------------

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:09:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:09:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stoc

Training Accuracy: 0.8650
Testing Accuracy:  0.8553
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.93      0.93       265
           1       0.78      0.82      0.80       260
           2       0.85      0.81      0.83       249

    accuracy                           0.86       774
   macro avg       0.86      0.85      0.85       774
weighted avg       0.86      0.86      0.86       774

------------------------------------------------------------

LIME Explanations for Test Instance (First Sample, original features):
Instance values: [ 2.27361605e-01 -4.91949618e-02  9.42274034e-02 -1.35573804e-01
 -1.22863680e-01 -1.63560539e-01  1.57175645e-01  6.39255270e-02
  7.28624389e-02  2.32553110e-03 -1.12150244e-01  3.02667648e-01
  6.47887588e-02  1.19132608e-01 -1.35609612e-01  1.80780012e-02
  4.50311273e-01  4.24533635e-02  7.81470761e-02 -1.94882363e-01
 -6.72378093e-02 -3.21377814e-01 -1.02093697e-01 -2.93038815e-01
 -1.

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:16:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost: 2

Explaining predictions for XGBoost...
------------------------------------------------------------
Stacking (All Models): Skipped LIME explanation (complex pipeline)


using smot dataset and NCA is not used


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import pandas as pd

# Optional: XGBoost
try:
    from xgboost import XGBClassifier
    xgb_available = True
except ImportError:
    xgb_available = False

# Load dataset
df = pd.read_excel("final_student_embeddings_upsampled.xlsx")

# Features and Labels
X = df.iloc[:, :768].values
y = df["Numerical_Label"]  # No label encoding

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Tune KNN
knn_param_grid = {'n_neighbors': list(range(1, 21))}
knn_grid = GridSearchCV(KNeighborsClassifier(), knn_param_grid, cv=5)
knn_grid.fit(X_train, y_train)
best_knn = knn_grid.best_estimator_

# Base models
base_models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(kernel='linear', probability=True, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=6, min_samples_split=10, min_samples_leaf=5, random_state=42),
    "Naive Bayes": GaussianNB(),
    "Decision Tree": DecisionTreeClassifier(max_depth=5, min_samples_split=10, min_samples_leaf=5, random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42),
    "MLP": MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', max_iter=500, random_state=42),
    "KNN": best_knn
}

if xgb_available:
    base_models["XGBoost"] = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Stacking Classifier using all models
stacking_estimators = [(name.lower().replace(" ", "_"), model) for name, model in base_models.items()]
stacking_clf = StackingClassifier(
    estimators=stacking_estimators,
    final_estimator=LogisticRegression(max_iter=1000, random_state=42),
    cv=5
)

# Add stacking model
models = base_models.copy()
models["Stacking (All Models)"] = stacking_clf

# Train & Evaluate
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    train_acc = accuracy_score(y_train, model.predict(X_train))
    test_acc = accuracy_score(y_test, y_pred)

    print(f"📌 {name}")
    print(f"Training Accuracy: {train_acc:.4f}")
    print(f"Testing Accuracy:  {test_acc:.4f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("-" * 60)


📌 Logistic Regression
Training Accuracy: 0.8385
Testing Accuracy:  0.5956
Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.78      0.72       265
           1       0.53      0.43      0.48       260
           2       0.56      0.57      0.57       249

    accuracy                           0.60       774
   macro avg       0.59      0.59      0.59       774
weighted avg       0.59      0.60      0.59       774

------------------------------------------------------------
📌 SVM
Training Accuracy: 0.9109
Testing Accuracy:  0.6382
Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.88      0.78       265
           1       0.59      0.45      0.51       260
           2       0.60      0.57      0.59       249

    accuracy                           0.64       774
   macro avg       0.63      0.64      0.63       774
weighted avg       0.63      0.64      0.63       774


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:36:51] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


📌 XGBoost
Training Accuracy: 1.0000
Testing Accuracy:  0.6537
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.88      0.83       265
           1       0.57      0.47      0.51       260
           2       0.57      0.61      0.59       249

    accuracy                           0.65       774
   macro avg       0.64      0.65      0.65       774
weighted avg       0.65      0.65      0.65       774

------------------------------------------------------------


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:41:56] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:52:19] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:53:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:54:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:55:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

📌 Stacking (All Models)
Training Accuracy: 1.0000
Testing Accuracy:  0.6835
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       265
           1       0.57      0.49      0.53       260
           2       0.60      0.65      0.62       249

    accuracy                           0.68       774
   macro avg       0.68      0.68      0.68       774
weighted avg       0.68      0.68      0.68       774

------------------------------------------------------------


normal dataset without using NCA and SMOT

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import pandas as pd

# Optional: XGBoost
try:
    from xgboost import XGBClassifier
    xgb_available = True
except ImportError:
    xgb_available = False

# Load dataset
df = pd.read_excel("student_embeddings_output.xlsx")

# Features and Labels
X = df.iloc[:, :768].values
y = df["Numerical_Label"]  # No label encoding

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Tune KNN
knn_param_grid = {'n_neighbors': list(range(1, 21))}
knn_grid = GridSearchCV(KNeighborsClassifier(), knn_param_grid, cv=5)
knn_grid.fit(X_train, y_train)
best_knn = knn_grid.best_estimator_

# Base models
base_models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(kernel='linear', probability=True, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=6, min_samples_split=10, min_samples_leaf=5, random_state=42),
    "Naive Bayes": GaussianNB(),
    "Decision Tree": DecisionTreeClassifier(max_depth=5, min_samples_split=10, min_samples_leaf=5, random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42),
    "MLP": MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', max_iter=500, random_state=42),
    "KNN": best_knn
}

if xgb_available:
    base_models["XGBoost"] = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Stacking Classifier using all models
stacking_estimators = [(name.lower().replace(" ", "_"), model) for name, model in base_models.items()]
stacking_clf = StackingClassifier(
    estimators=stacking_estimators,
    final_estimator=LogisticRegression(max_iter=1000, random_state=42),
    cv=5
)

# Add stacking model
models = base_models.copy()
models["Stacking (All Models)"] = stacking_clf

# Train & Evaluate
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    train_acc = accuracy_score(y_train, model.predict(X_train))
    test_acc = accuracy_score(y_test, y_pred)

    print(f"📌 {name}")
    print(f"Training Accuracy: {train_acc:.4f}")
    print(f"Testing Accuracy:  {test_acc:.4f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("-" * 60)


📌 Logistic Regression
Training Accuracy: 0.8447
Testing Accuracy:  0.4112
Classification Report:
              precision    recall  f1-score   support

           0       0.26      0.26      0.26        95
           1       0.46      0.47      0.46       258
           2       0.42      0.41      0.42       216

    accuracy                           0.41       569
   macro avg       0.38      0.38      0.38       569
weighted avg       0.41      0.41      0.41       569

------------------------------------------------------------
📌 SVM
Training Accuracy: 0.9041
Testing Accuracy:  0.4165
Classification Report:
              precision    recall  f1-score   support

           0       0.26      0.38      0.31        95
           1       0.47      0.43      0.45       258
           2       0.46      0.41      0.43       216

    accuracy                           0.42       569
   macro avg       0.40      0.41      0.40       569
weighted avg       0.43      0.42      0.42       569


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


📌 Decision Tree
Training Accuracy: 0.5974
Testing Accuracy:  0.4376
Classification Report:
              precision    recall  f1-score   support

           0       0.37      0.12      0.18        95
           1       0.45      0.67      0.54       258
           2       0.41      0.31      0.35       216

    accuracy                           0.44       569
   macro avg       0.41      0.36      0.36       569
weighted avg       0.42      0.44      0.41       569

------------------------------------------------------------
📌 AdaBoost
Training Accuracy: 0.5381
Testing Accuracy:  0.4552
Classification Report:
              precision    recall  f1-score   support

           0       0.46      0.14      0.21        95
           1       0.46      0.61      0.52       258
           2       0.45      0.41      0.43       216

    accuracy                           0.46       569
   macro avg       0.46      0.39      0.39       569
weighted avg       0.46      0.46      0.44       569



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:59:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


📌 XGBoost
Training Accuracy: 1.0000
Testing Accuracy:  0.4534
Classification Report:
              precision    recall  f1-score   support

           0       0.30      0.11      0.16        95
           1       0.48      0.59      0.53       258
           2       0.44      0.44      0.44       216

    accuracy                           0.45       569
   macro avg       0.41      0.38      0.38       569
weighted avg       0.43      0.45      0.43       569

------------------------------------------------------------


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:03:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:11:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:11:52] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:12:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:13:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

📌 Stacking (All Models)
Training Accuracy: 0.9912
Testing Accuracy:  0.4780
Classification Report:
              precision    recall  f1-score   support

           0       0.32      0.14      0.19        95
           1       0.48      0.66      0.56       258
           2       0.51      0.41      0.45       216

    accuracy                           0.48       569
   macro avg       0.44      0.40      0.40       569
weighted avg       0.46      0.48      0.46       569

------------------------------------------------------------
